# Question 1: 8-Puzzle Game

#### The 8-Puzzle Problem Overview:
The 8-puzzle game consists of a 3x3 grid with 9 squares and 8 numbered tiles. The game starts with the 8 numbered tiles placed in their 'Initial State'- the player shifts one tile at a time in an effort to match the tile placement to the grid's 'Goal State'.
The game rules are as follows:
- Goal: reach the tile configuration in the 'Goal State' from the 'Initial State' 
- Legal Actions: up, down, left, right. 
    - One tile at a time. 
    - Only tiles adjacent to the blank space in the current state. 

This game can be viewed as a search problem in the sense that the player is searching for a path that would lead from the 'Initial State' to the 'Goal State' by rearranging the tiles.
- For example: The paths would be considered short if *few tiles* are moved and *few moves* are made in the process of reaching the goal state - and vice versa.
- An example of a task that applies the same logic is mapping directions to a location (ex. GPS). 


The A* algorithm is implemented using Python to solve the given 8-Puzzle problem.

#### A* Algorithm Overview:
Heuristics are *informed* problem-solving techniques, meaning that they use a heuristic / estimation of the fastest path to the solution. This can provide a time-efficient solution to given problems. The solution / path is based on a simple set of rules (depending on the problem), however, using a heuristic function does not guarantee that the solution is optimal, just that it is complete.
The A* algorithm is a computer algorithm that employs heuristic functions to find the path in a tree with the lowest cost  from the start node to the goal node. With an admissible heuristic function, A* provides a complete and optimal solution. A heuristic function is considered admissible if it uses an optimistic estimate- that is that it does not overestimate the cost. The algorithm keeps track of each node visited and doesn't waste time exploring paths with relatively higher cost. 
The A* algorithm uses the following function f(n) to calculate the cost of a move at every node:
*f(n) = g(n) + h(n)*
- *f(n)* = total estimated cost of path through node *n*
- *g(n)* = sum of costs so far that made us reach node *n*
- *h(n)* =  estimated cost from node *n* to goal node (heuristic)
    - Can be calculated using Manhattan Distance from node *n* to the goal node. Manhattan Distance is the sum of vertical and horizontal steps / distance between two points.
    - Ideally *h(n) = h x (n)*
    - When *h(n)* = 0, the algorithm prioritizes finding the *shortest* path to the goal node (aka Dijkstra's algorithm)
 
 
 The goal is to choose the path with the smallest cost (*f(n)* ) possible. 

#### Admissible heuristic functions:
1. *h<sub>1</sub>(n)*: number of misplaced tiles
2. *h<sub>2</sub>(n)*: sum of Manhattan distance for every tile from current to goal state


The reason these two heuristics are admissible is because they do not overestimate the cost of moves made to reach the goal state, and help us find an optimal path from the start to the goal state. They help guide the algorithm from the start state to the goal state efficiently. 
The reason H1 was chosen is because every tile in the start state is misplaced and will therefore need to be moved in order to reach the goal state. 
As for H2, it is used since with every move each tile can only get 1 step closer to the goal position. Our goal is to minimize both functions; the smaller the sum of misplaced tiles and the smaller the sum of distances, the closer we are to reaching the goal state.  

### 8-Puzzle Solver: Implementing A* Algorithm

In [79]:
# Import required libraries
from math import sqrt
from collections import deque
from heapq import heappush, heappop
import time

In [80]:
class State:
    # Defining the state of the puzzle
    def __init__(self, state, parent, move, depth, cost, key):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.key = key
        if self.state:
            self.map = ''.join(str(e) for e in self.state)
    def __eq__(self, other):
        return self.map == other.map
    def __lt__(self, other):
        return self.map < other.map

In [81]:
# Initializing goal state and conditions
goalState = [1,2,3,4,5,6,7,8,0]
goalNode = State
initialState = []   
puzzleLen = 9       
puzzleSide = 0      
nodesExpanded = 0   
maxDepthReached = 0 
maxFringeSize = 0   
moves = []         

In [82]:
def astar(startState, heuristicFunc):
    # A star algorithm function
    global goalNode, maxFringeSize, maxDepthReached
    visited, pQueue = set(), list()
    key = heuristicFunc(startState)
    root = State(startState, None, None, 0, 0, key)
    entry = (key, 0, root)
    heappush(pQueue, entry)
    while pQueue:
        node = heappop(pQueue)
        visited.add(node[2].map)
        if node[2].state == goalState:
            goalNode = node[2]
            return True, pQueue
        neighbors = expand(node[2])
        for neighbor in neighbors:
            neighbor.key = neighbor.cost + heuristicFunc(neighbor.state)
            entry = (neighbor.key, neighbor.move, neighbor)
            if neighbor.map not in visited:
                heappush(pQueue, entry)
                visited.add(neighbor.map)
                if neighbor.depth > maxDepthReached:
                    maxDepthReached += 1                    
        if len(pQueue) > maxFringeSize:
            maxFringeSize = len(pQueue)
    return False, None

In [83]:
def h1(state):
    # Heuristic function 1: number of misplaced tiles
    count = 0
    for i in range(0, 9):
        if not (state.index(i) == goalState.index(i)) : 
            count+=1
    return count 

In [84]:
def h2(state): 
    # Heuristic function 2: sum of Manhattan distance for every tile from current to goal state
    return sum(abs(p%puzzleSide - g%puzzleSide) + abs(p//puzzleSide - g//puzzleSide)
               for p,g in ((state.index(i),goalState.index(i)) 
               for i in range(1, 9))) 

In [85]:
def expand(node):
    # Function to create valid child nodes in the tree
    global nodesExpanded
    nodesExpanded += 1
    childNodes = []
    childNodes.append(State(legalMove(node.state,'DOWN'),node,'DOWN',node.depth + 1, node.cost + 1, 0)) 
    childNodes.append(State(legalMove(node.state,'LEFT'),node,'LOW',node.depth + 1, node.cost + 1, 0)) 
    childNodes.append(State(legalMove(node.state,'RIGHT'),node,'RIGHT',node.depth + 1, node.cost + 1, 0))
    childNodes.append(State(legalMove(node.state,'UP'),node,'UP',node.depth + 1, node.cost + 1, 0))
    nodes = [child for child in childNodes if child.state]
    return nodes

In [86]:
def legalMove(state, position):
    # Function to check that the move is legal
    newState = state[:]
    index = newState.index(0) 
    if position == 'UP': 
        if index not in range(0, puzzleSide):           
            temp = newState[index - puzzleSide]
            newState[index - puzzleSide] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None
    if position == 'DOWN':        
        if index not in range(puzzleLen - puzzleSide, puzzleLen):
            temp = newState[index + puzzleSide]
            newState[index + puzzleSide] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None
    if position == 'LEFT': 
        if index not in range(0, puzzleLen, puzzleSide):
            temp = newState[index - 1]
            newState[index - 1] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None
    if position == 'RIGHT': 
        if index not in range(puzzleSide - 1, puzzleLen, puzzleSide):
            temp = newState[index + 1]
            newState[index + 1] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None

In [87]:
def get(dataList):
    # Function retrieve the elements in the input list
    global puzzleLen, puzzleSide
    data = dataList.split(',') 
    for element in data:
        initialState.append(int(element))
    puzzleLen = len(initialState)                  
    puzzleSide = int(sqrt(puzzleLen))

In [88]:
def backtrack():
    # Function to check the state of the nodes
    currentNode = goalNode
    while initialState != currentNode.state : 
        moves.insert(0, currentNode.move)
        currentNode = currentNode.parent
    return moves

In [89]:
def output(fringe, time, testNum):
    # Setting output
    if fringe:
        global moves
        moves = backtrack()
        print("<-- # SOLUTION DETAILS # -->")
        print("\n Initial State: "+str(initialState))
        print("\n Path to goal: " + str(moves))
        print("\n Cost(Moves): " + str(len(moves)))
        print("\n Node count: " + str(nodesExpanded))
        print("\n Fringe size: " + str(len(fringe)))
        print("\n Tree depth: " + str(goalNode.depth))
        print("\n Run time: " + format(time, '.8f'))
    else:
        print("<-- # NO SOLUTION # -->")
        print("\n Node count: " + str(nodesExpanded))
        print("\n Max fringe: " + str(maxFringeSize))
        print("\n Max tree depth: " + str(maxDepthReached))
        print("\n Run time: " + format(time, '.8f'))

In [90]:
def main():
    heuristic_map = {'h1' : h1,'h2' : h2}
    while True:
        heuristic = input('WELCOME TO 8-PUZZLE SOLVER USING A* ALGORITHM! \
        \n\
        \n--> Please select a heuristic function:\
        \n h1: number of misplaces tiles\
        \n h2: sum of Manhattan distance for every tile from current to goal state \
        \n-- Enter your choice: ')
        if(heuristic == 'h1' or heuristic == 'h2'):
            heuristicFunc = heuristic_map[heuristic]
            break
        print("Please type 'h1' or 'h2'.")
        
        
    data = input('--> Enter puzzle elements: ')    
    get(data)
    function = astar 
    start = time.time()       
    search, fringe = function(initialState, heuristicFunc)  
    stop = time.time()
    if not search : 
        output(fringe, stop-start, 'No_Solution')
    else : 
        output(fringe, stop-start, 'h2')

### Start Playing!

The user will get the choice to select one of the two heuristic functions, and will then be asked to input the elements of the puzzle.
This can solve any 8-puzzle and return the numbers to the correct ascending order (0,1,2,3,4,5,6,7,8). The 'Path to solution' refers to the movement of the EMPTY tile (0). 

In [ ]:
if __name__ == '__main__':
    main()

### Results
The example given in the coursework outline was solved. The details of the solution are as follows:
- Initial state: (7, 2, 4, 5, 0, 6, 8, 3, 1)
- Goal state: (0,1,2,3,4,5,6,7,8)
- Path to solution: ('DOWN', 'RIGHT', 'UP', 'LOW', 'LOW', 'UP', 'RIGHT', 'RIGHT', 'DOWN', 'LOW', 'DOWN', 'LOW', 'UP', 'RIGHT', 'UP', 'LOW', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN')


Our algorithm was able to successfuly solve both 8-puzzles using both heuristic functions. Both heuristic function resulted in the same cost: 20. However, the second heuristic (Manhattan sum of distances for every tile from current to goal state), seemed to perform better relatively, with a significantly smaller number of expanded nodes and fringe, and a slightly shorter run time. The fringe tells us the number of possible nodes we can move to from the current state. The smaller fringe size could indicate that the tree was better at choosing next moves and eliminating incorrect states. The performance of the second heuristic function could also be explained by the fact that it acts as a better guide to the correct path leading to the solution. The estimate is more useful for finding the correct placement of each individual tile. 


The following table compares the results of both heuristics:

|.| H1 (misplaced tiles) | H2 (sum of distances) |
| --- | --- | --- |
| cost | 20 | 20 |
| nodes| 3495 | 204 |
| fringe | 1888 | 129 | 
| depth | 20 | 20 |
| run time| 0.144 | 0.011|

# Question 2: Sudoko Puzzle

#### Evolutionary Algorithm Overview:
Evolutionary algorithms are algorithms inspired by Darwin’s Theory of Evolution that use mechanisms similar to those that occur in nature. The algorithms aim to mimic and apply behaviors of organisms computationally in an effort to create effective solutions to different problems.  
Much like living biological creatures, Evolutionary Algorithms revolve around the concept of ‘survival of the fittest’ and include functions like selection, reproduction, mutation  and  recombination. 


The subset of the Evolutionary Algorithm we will be using to solve the Sudoku puzzle is the Genetic Algorithm - a heuristic search algorithm based on genetics and natural selection. 

#### Genetic Algorithm Overview:
The Genetic Algorithm mimics the process of natural selection and 'survival of the fittest', where the best individual is selected for reproduction of the next generation. Given a population, the most fit 'parents' are selected to reproduce. The 'children' then inherit the 'genes' (characteristics) of the parent, and the process repeats until we are left with the most fit population of individuals. 


This could be implemented to search problems by eliminating the least fit solutions and selecting the best one. The five phases we consider are:
1. Initial population: This is the set of solutions. Each individual is characterized by a set of 'genes' (parameters) that form a 'chromosome' (solution). 
2. Fitness function: This computes the probability of an 'individual' (solution) being more fit relative to the others in a population. 
3. Selection: The most fit 'individuals' (solutions) are selected and paired to be 'parents' and are responsible for creating offspring and passing on their 'genes' (parameters).
4. Crossover: This is the most important phase of the algorithm, where a *crossover point* is selected at random for both 'parents genes', and the 'genes' are exchanged. This creates an offspring with the characteristics of both 'parents', and the offspring is added to the population (as a new, fit soltion).
5. Mutation: The purpose of this phase is to increase diversity. In this phase, some of the 'genes' positions are randomly swapped. 


##### When does it end?


The process terminates when the offspring produced doesn't show significant change / imporvement in fitness. This means that we have reached optimal fitness (aka the best solution). 

### Sudoku Puzzle Overview & Design:
The Sudoku puzzle is a popular game consisting of a 9x9 grid made up of 9 smaller 3x3 squares. The goal of the game is to fill out the grid with numbers 1-9 without repeating numbers in each column, row, or square. The numbers in each column, row, and square should sum up to 45. 



We implement the Genetic Algorithm to create a program that can solve Sudoku puzzles in a computationally efficient manner. We use the five phases and design the algorithm as follows:


1. **Initial population:** Generate a random permutation of numbers between 1-9 to fill empty spaces in the grid.

2. **Fitness function:**: We use a constraint satisfaction fitness function to evaluate the fitness of each generated solution. The solution with the least duplicates that best satisfies the constraints is selected for reproduction. The function measures the number of duplicates in each row, column, and square. Fewer duplicates = better fitness The constraints of the Sudoku puzzle are:
    - Each *row* should not have duplicates
    - Each *column* should not have duplicates
    - Each *square* should not have duplicates
    
The simplest way to represent a fitness function for the Sudoku puzzle is as follows:


*fitness function* = $\sum_{i=1}^{9} r_i + c_i + s_i$


where:
- $r_i$ = duplicate numbers in row i
- $c_i$ = duplicate numbers in column i
- $c_i$ = duplicate numbers in subgrid i


However, in our program, we use a *linear combination of the partial costs* to create the *constraint satisfaction fitness function* used by the algorithm. This helps us ensure that the solution satisfies the requirements of the game and allows us to effectively measure its fitness. 

3. **Selection:** The selection process is done using *ranking* and *tournament* operators. 
    - Ranking: ranks all the solutions in a population based on fitness (1 being the most fit, etc...)
    - Tournament: randomly selects solutions from a population and compares their fitness. The solutions with higher fitness scores are reserved, while the solutions with lower fitness scores 'lose' the tournament and are eliminated. 
    
4. **Crossover**: In 9x9 Sudoku puzzles, there are 8 possible *crossover points*. The crossover operators we used are: 
    - One-point crossover: *one* point is selected for both 'parents', and the position of the numbers in the offpspring is flipped accordingly.
    - Uniform genes crossover: *randomly* selects 'gene' from the 'parents' for the offspring to inherit.
    
5. **Mutation**: We use *swap* as a mutation operator, which causes two numbers in a subgrid to swich positions. We set a small value for the mutation rate (0.01) for all population sizes, as a high value will lead to increased randomness. We also use *reset* mutation, which randomly changes one of the gene's values to a different legal value (range 1-9).

6. **Termination**: The process will terminate once the constraints are satisified and the puzzle is solved. Otherwise, if the offspring shows no improvement in fitness, the puzzle is ruled to have no solution. 

* NOTE 1: in the code, we use the term 'Alleles', 'Phenotype', and 'Genotype'. Alleles are variant forms of genes. In the case of our puzzle, it refers to the value held inside the 'gene'. 'Phenotype' refers to the physical representation of the chromosome, while 'Genotype' refers to the set of *genes* representing the chromosome. 
* NOTE 2: One of the features of this algorithm is *Elitism*, which duplicates the fittest solutions to prevent them from being discarded throughout the process and ensures that they are reproducing and passing their genes on, ensuring that the children nodes are improving in fitness. 

### Sudoku Puzzle Solver: Implementing the Genetic Algorithm

In [1]:
import random, math, operator, time
import sys, os

class SizeExtension(object):
    def __init__ (self, size):
        self.size = size
    def __len__(self):
        return self.size

class Allele(object):
    def symbols (size):
        return range(1, size + 1)

class Gene(SizeExtension):
    
    # Dictionary to access mutation methods
    mutationMethods = dict(Reset = 'resetMutation', Swap  = 'swapMutation')

    def __init__ (self, candidate):
        super(Gene, self).__init__(len(candidate))

        self.candidate = candidate
        self.symbols = Allele.symbols(self.size)
        self.missing = list(set(self.symbols) - set(self.candidate))
        self.alleles = []

    def randomize(self):
        # Randomly fill empty squares
        missing = list(self.missing)
        random.shuffle(missing)
        self.alleles = list(self.candidate)
        indexes = [i for i, x in enumerate(self.alleles) if x == 0]
        
        for index in indexes:
            self.alleles[index] = missing.pop()

    def swapMutation(self):
        # Swap mutation switches the position of numbers
        minimumNumberOfPoints = 2
        allPoints = [i for i, x in enumerate(self.candidate) if x == 0]
        numberOfAllPoints = len(allPoints)
        if (numberOfAllPoints < minimumNumberOfPoints): return
        randomNumberOfPoints = random.randrange(minimumNumberOfPoints, numberOfAllPoints + 1)
        points = random.sample(allPoints, randomNumberOfPoints)
        points.sort (reverse = (True if (random.random() < 0.5) else False))
        for i in points:
            xorValue = 0
            for j in points:
                xorValue ^= self.alleles[j]
            self.alleles[i] = xorValue
        xorValue = 0
        for point in points:
            xorValue ^= self.alleles[point]
        self.alleles[points[0]] = xorValue
        
        
    def resetMutation(self):
        # Reset mutation randomly assigns value to gene
        self.randomize()
        
    def mutate(self):
        key = random.choice(list(Gene.mutationMethods.keys()))
        getattr (self, Gene.mutationMethods[key])()


class Genotype(SizeExtension):
    
    # Dictionary containing crossover methods
    crossoverMethods = dict(OnePoint = 'onePointCrossover', UniformGene = 'uniformGeneCrossover')

    def __init__ (self, candidate):
        super(Genotype, self).__init__(len(candidate))
        self.candidate = candidate
        self.symbols = Allele.symbols(self.size)
        self.genes = []

    def randomize (self):
        self.genes = []
        for gene in range(self.size):
            g = Gene(self.candidate[gene])
            g.randomize()
            self.genes.append(g)
          
    def set(self, genotype):
        self.genes = genotype.genes

    def onePointCrossover(self, other):
        # Function for one point cross over 
        # One point is selected for both 'parents', and the position of the 
        # numbers in the children is flipped accordingly
        point = random.randrange(1, self.size)
        children = []
        children.append(Genotype(self.candidate))
        children.append(Genotype(self.candidate))
        children[0].genes = self.genes[:point] + other.genes[point:]
        children[1].genes = other.genes[:point] + self.genes[point:]
        return children  
    
    def uniformGeneCrossover(self, other):
        # Function for the uniform gene crossover
        # Randomly selects 'gene' from the 'parents' for the child to inherit
        children = []
        children.append(Genotype(self.candidate))
        children.append(Genotype(self.candidate))
        for gene in range(self.size):
            if (random.random() < 0.5):
                children[0].genes.append(self.genes[gene])
                children[1].genes.append(other.genes[gene])
            else:
                children[0].genes.append(other.genes[gene])
                children[1].genes.append(self.genes[gene])
        return children
    
    def crossover (self, other):
        key = random.choice(list(Genotype.crossoverMethods.keys()))
        return getattr (self, Genotype.crossoverMethods[key])(other)
    
    def mutate (self):
        gene = random.randrange(0, self.size)
        self.genes[gene].mutate()


class Phenotype(SizeExtension):
    
    ranges = {'9': [ range(0, 3), range(3, 6), range(6, 9) ]}
    slices = {'9': [ slice(0, 3), slice(3, 6), slice(6, 9) ]}
 
    def __init__ (self, genotype):
        super(Phenotype, self).__init__(len(genotype))
        self.genotype = genotype
        self.table = []

        
    def getRanges(self):
        try:
            return Phenotype.ranges[str(self.size)]
        except KeyError:
            pass

    def getSlices(self):
        try:
            return Phenotype.slices[str(self.size)]
        except KeyError:
            pass

    def __str__(self):
        return str(self.table)

    def convert (self):
        self.table = []
        ranges = self.getRanges()
        slices = self.getSlices()
        for r, rng in enumerate(ranges):
            for s, slc in enumerate(slices):
                row = []
                for i in rng:
                    row.extend(self.genotype.genes[i].alleles[slc])
                self.table.append(row)

                
class Individual(object):
# Each solution in a population
    
    def __init__ (self, candidate):
        self.genotype = Genotype(candidate)

    def randomize (self):
        self.genotype.randomize()

    def phenotype(self):
        phenotype = Phenotype(self.genotype)
        phenotype.convert()
        return phenotype

    def crossover (self, other):
        offsprings = self.genotype.crossover(other.genotype)
        children = []
        for offspring in offsprings:
            child = Individual(self.genotype.candidate)
            child.genotype.set(offspring)
            children.append(child)
        return children

    def mutate (self):
        self.genotype.mutate()

    def estimate (self):
        phenotype = self.phenotype()
        size = len(phenotype)
        symbols = self.genotype.symbols
        rowsSumDistance = [0] * size
        colsSumDistance = [0] * size
        rowsPrdDistance = [1] * size
        colsPrdDistance = [1] * size
        rowsMissed = [0] * size
        colsMissed = [0] * size
        symbolsSum = sum(symbols)
        symbolsFac = math.factorial(len(symbols))
        symbolsSet = set(symbols)
        for i in range(size):
            for j in range(size):
                rowsSumDistance[i] += phenotype.table[i][j]
                rowsPrdDistance[i] *= phenotype.table[i][j]
                colsSumDistance[i] += phenotype.table[j][i]
                colsPrdDistance[i] *= phenotype.table[j][i]
            rowsSumDistance[i] = abs(symbolsSum - rowsSumDistance[i])
            rowsPrdDistance[i] = abs(symbolsFac - rowsPrdDistance[i])
            colsSumDistance[i] = abs(symbolsSum - colsSumDistance[i])
            colsPrdDistance[i] = abs(symbolsFac - colsPrdDistance[i])
            rowsMissed[i] = len(list(symbolsSet - set(phenotype.table[i])))
            colsMissed[i] = len(list(symbolsSet - set([row[i] for row in phenotype.table])))
        rootsColsPrd = [math.sqrt(i) for i in colsPrdDistance]
        rootsRowsPrd = [math.sqrt(i) for i in rowsPrdDistance]

        # fitness func
        self.fitness = (10 * (sum(rowsSumDistance) + sum(colsSumDistance)) 
                        + 50 * (sum(rowsMissed) + sum(colsMissed)) 
                        + sum(rootsColsPrd) + sum(rootsRowsPrd))


class Population(SizeExtension):
# Set of solutions
    
    # Dictionary for accessing selection methods
    selectionMethods = dict(Tournament = 'tournamentSelection', Rank = 'rankSelection')

    def __init__ (self, populationSize, tournamentSize, gridGenotype):
        super(Population, self).__init__(populationSize)
        self.tournamentSize = tournamentSize
        self.gridGenotype = gridGenotype
        self.strongest = None

        self.parents = []
        self.children = []

    def initialize(self):
        self.parents = []
        for i in range(self.size):
            individual = Individual(self.gridGenotype)
            individual.randomize()
            self.parents.append(individual)

    def estimate(self):
        [parent.estimate() for parent in self.parents]

    def resolved(self):
        self.strongest = min(self.parents, key = lambda o: o.fitness)
        return (self.strongest.fitness == 0)

    def generation(self):
        self.children = []

    def elitism(self):
        self.children.append(self.strongest)

    def fitnessOfStrongest(self):
        return self.strongest.fitness

    def reproduced(self):
        return (len(self.children) >= len(self.parents))

    def select(self):
        key = random.choice(list(Population.selectionMethods.keys()))
        return getattr (self, Population.selectionMethods[key])()

    def addChildren(self, children):
        for child in children:
            if self.reproduced():
                break
        else:
                self.children.append(child)

    def replacement(self):
        self.parents = self.children

    def rankSelection(self):
        parents = sorted (self.parents, key = lambda o: o.fitness)
        ranks = range(1, self.size + 1)
        sumRanks = sum(ranks)
        for i, parent in enumerate(parents):
            parent.probability = (((self.size - ranks[i] + 1) / float(sumRanks)) * 100)
        pick = random.uniform(0, sum([p.probability for p in parents]))
        current = 0
        for parent in parents:
            current += parent.probability
            if current > pick:
                return parent

    def tournamentSelection(self):
        indexes = [random.randrange(0, self.size) for i in range(self.tournamentSize)]
        return min([self.parents[i] for i in indexes], key = lambda o: o.fitness)

#### Random seed set to 1234

In [2]:
class SudokuSolver(object):
    def __init__ (self, parameters):
        
        self.mutationProbability = parameters['mutationProbability']
        self.populationSize = parameters['populationSize']
        self.tournamentSize = parameters['tournamentSize']
        self.gridGenotype = parameters['gridGenotype']
        self.population = Population (self.populationSize, self.tournamentSize, self.gridGenotype)
        random.seed (1234)

    def evolve(self):
        self.population.initialize()
        self.population.estimate()
        while (not self.population.resolved()):
            self.population.generation()
            self.population.elitism()
            self.population.fitnessOfStrongest()
            while (not self.population.reproduced()):
                mother = self.population.select()
                father = self.population.select()
                children = mother.crossover(father)
                for child in children:
                    if (random.random() < self.mutationProbability):
                        child.mutate()
                    child.estimate()
                self.population.addChildren(children)
            self.population.replacement()
        return self.population.strongest.phenotype()

### Population size set to 10

##### Puzzle 1: Population size 10

In [ ]:
parameters = {
    
    'populationSize': 10,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[3, 0, 0, 0, 0, 5, 0, 4, 7],
                  [0, 0, 6, 0, 4, 2, 0, 0, 1],
                  [0, 0, 0, 0, 0, 7, 8, 9, 0],
                  [0, 5, 0, 0, 1, 6, 0, 0, 2],
                  [0, 0, 3, 0, 0, 0, 0, 0, 4],
                  [8, 1, 0, 0, 0, 0, 7, 0, 0],
                  [0, 0, 2, 0, 0, 0, 4, 0, 0],
                  [5, 6, 0, 8, 7, 0, 1, 0, 0],
                  [0, 0, 0, 3, 0, 0, 6, 0, 0]]
}



import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

##### Puzzle 2: Population size 10

In [ ]:
parameters = {
    
    'populationSize': 10,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[0, 0, 2, 0, 0, 0, 6, 3, 4],
                  [1, 0, 6, 0, 0, 0, 5, 8, 0],
                  [0, 0, 7, 3, 0, 0, 2, 9, 0],
                  [0, 8, 5, 0, 0, 1, 0, 0, 6],
                  [0, 0, 0, 7, 5, 0, 0, 2, 3],
                  [0, 0, 3, 0, 0, 0, 0, 5, 0],
                  [3, 1, 4, 0, 0, 2, 0, 0, 0],
                  [0, 0, 9, 0, 8, 0, 4, 0, 0],
                  [7, 2, 0, 0, 4, 0, 0, 0, 9]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

##### Puzzle 3: Population size 10

In [ ]:
parameters = {
    
    'populationSize': 10,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[0, 0, 4, 0, 1, 0, 0, 6, 0],
                  [9, 0, 0, 0, 0, 0, 0, 3, 0],
                  [0, 5, 0, 7, 9, 6, 0, 0, 0],
                  [0, 0, 2, 5, 0, 4, 9, 0, 0],
                  [0, 8, 3, 0, 6, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 6, 0, 7],
                  [0, 0, 0, 9, 0, 3, 0, 7, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 6, 0, 0, 0, 0, 1, 0]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

### Population size set to 100

##### Puzzle 1: Population size 100

In [ ]:
parameters = {
    
    'populationSize': 100,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[3, 0, 0, 0, 0, 5, 0, 4, 7],
                  [0, 0, 6, 0, 4, 2, 0, 0, 1],
                  [0, 0, 0, 0, 0, 7, 8, 9, 0],
                  [0, 5, 0, 0, 1, 6, 0, 0, 2],
                  [0, 0, 3, 0, 0, 0, 0, 0, 4],
                  [8, 1, 0, 0, 0, 0, 7, 0, 0],
                  [0, 0, 2, 0, 0, 0, 4, 0, 0],
                  [5, 6, 0, 8, 7, 0, 1, 0, 0],
                  [0, 0, 0, 3, 0, 0, 6, 0, 0]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

##### Puzzle 2: Population size 100

In [ ]:
parameters = {
    
    'populationSize': 100,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[0, 0, 2, 0, 0, 0, 6, 3, 4],
                  [1, 0, 6, 0, 0, 0, 5, 8, 0],
                  [0, 0, 7, 3, 0, 0, 2, 9, 0],
                  [0, 8, 5, 0, 0, 1, 0, 0, 6],
                  [0, 0, 0, 7, 5, 0, 0, 2, 3],
                  [0, 0, 3, 0, 0, 0, 0, 5, 0],
                  [3, 1, 4, 0, 0, 2, 0, 0, 0],
                  [0, 0, 9, 0, 8, 0, 4, 0, 0],
                  [7, 2, 0, 0, 4, 0, 0, 0, 9]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

##### Puzzle 3: Population size 100

In [ ]:
parameters = {
    
    'populationSize': 100,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[0, 0, 4, 0, 1, 0, 0, 6, 0],
                  [9, 0, 0, 0, 0, 0, 0, 3, 0],
                  [0, 5, 0, 7, 9, 6, 0, 0, 0],
                  [0, 0, 2, 5, 0, 4, 9, 0, 0],
                  [0, 8, 3, 0, 6, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 6, 0, 7],
                  [0, 0, 0, 9, 0, 3, 0, 7, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 6, 0, 0, 0, 0, 1, 0]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

### Population size set to 1,000

##### Puzzle 1: Population size 1000

In [ ]:
parameters = {
    
    'populationSize': 1000,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[3, 0, 0, 0, 0, 5, 0, 4, 7],
                  [0, 0, 6, 0, 4, 2, 0, 0, 1],
                  [0, 0, 0, 0, 0, 7, 8, 9, 0],
                  [0, 5, 0, 0, 1, 6, 0, 0, 2],
                  [0, 0, 3, 0, 0, 0, 0, 0, 4],
                  [8, 1, 0, 0, 0, 0, 7, 0, 0],
                  [0, 0, 2, 0, 0, 0, 4, 0, 0],
                  [5, 6, 0, 8, 7, 0, 1, 0, 0],
                  [0, 0, 0, 3, 0, 0, 6, 0, 0]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

##### Puzzle 2: Population size 1000

In [ ]:
parameters = {
    
    'populationSize': 1000,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[0, 0, 2, 0, 0, 0, 6, 3, 4],
                  [1, 0, 6, 0, 0, 0, 5, 8, 0],
                  [0, 0, 7, 3, 0, 0, 2, 9, 0],
                  [0, 8, 5, 0, 0, 1, 0, 0, 6],
                  [0, 0, 0, 7, 5, 0, 0, 2, 3],
                  [0, 0, 3, 0, 0, 0, 0, 5, 0],
                  [3, 1, 4, 0, 0, 2, 0, 0, 0],
                  [0, 0, 9, 0, 8, 0, 4, 0, 0],
                  [7, 2, 0, 0, 4, 0, 0, 0, 9]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

##### Puzzle 3: Population size 1000

In [ ]:
parameters = {
    
    'populationSize': 10,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[0, 0, 4, 0, 1, 0, 0, 6, 0],
                  [9, 0, 0, 0, 0, 0, 0, 3, 0],
                  [0, 5, 0, 7, 9, 6, 0, 0, 0],
                  [0, 0, 2, 5, 0, 4, 9, 0, 0],
                  [0, 8, 3, 0, 6, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 6, 0, 7],
                  [0, 0, 0, 9, 0, 3, 0, 7, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 6, 0, 0, 0, 0, 1, 0]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

### Population size set to 10,000

##### Puzzle 1: Population size 10000

In [ ]:
parameters = {
    
    'populationSize': 10000,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[3, 0, 0, 0, 0, 5, 0, 4, 7],
                  [0, 0, 6, 0, 4, 2, 0, 0, 1],
                  [0, 0, 0, 0, 0, 7, 8, 9, 0],
                  [0, 5, 0, 0, 1, 6, 0, 0, 2],
                  [0, 0, 3, 0, 0, 0, 0, 0, 4],
                  [8, 1, 0, 0, 0, 0, 7, 0, 0],
                  [0, 0, 2, 0, 0, 0, 4, 0, 0],
                  [5, 6, 0, 8, 7, 0, 1, 0, 0],
                  [0, 0, 0, 3, 0, 0, 6, 0, 0]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

##### Puzzle 2: Population size 10000

In [ ]:
parameters = {
    
    'populationSize': 10000,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[0, 0, 2, 0, 0, 0, 6, 3, 4],
                  [1, 0, 6, 0, 0, 0, 5, 8, 0],
                  [0, 0, 7, 3, 0, 0, 2, 9, 0],
                  [0, 8, 5, 0, 0, 1, 0, 0, 6],
                  [0, 0, 0, 7, 5, 0, 0, 2, 3],
                  [0, 0, 3, 0, 0, 0, 0, 5, 0],
                  [3, 1, 4, 0, 0, 2, 0, 0, 0],
                  [0, 0, 9, 0, 8, 0, 4, 0, 0],
                  [7, 2, 0, 0, 4, 0, 0, 0, 9]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

##### Puzzle 3: Population size 10000

In [ ]:
parameters = {
    
    'populationSize': 10,

    'mutationProbability': 0.01,

    'tournamentSize': 8,

    'gridGenotype': [[0, 0, 4, 0, 1, 0, 0, 6, 0],
                  [9, 0, 0, 0, 0, 0, 0, 3, 0],
                  [0, 5, 0, 7, 9, 6, 0, 0, 0],
                  [0, 0, 2, 5, 0, 4, 9, 0, 0],
                  [0, 8, 3, 0, 6, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 6, 0, 7],
                  [0, 0, 0, 9, 0, 3, 0, 7, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 6, 0, 0, 0, 0, 1, 0]]
}


import timeit

start = timeit.default_timer()

ga = SudokuSolver(parameters)

solution = ga.evolve()

stop = timeit.default_timer()


print(solution)
print('Time: ', stop - start)

## Results and Analysis

The implemented genetic algorithm was able to successfully solve the Sudoku Puzzles, however, the run time was not ideal. The program was not as time-efficient as predicted. The best population size for the Sudoku puzzle would be either 100 or 10000, as they are both perfect squares. The worst population size was 10, which makes sense since 10 is a small population size, and the larger the population is, the higher the likelihood of it containig an optimal solution.

As for further improvement, I would try to use the truncation rate as a selection method along with the ranking and tournament. The truncation rate would order solutions based on fitness score, and discard the percentage of solutions that are least fit, and keep the fittest solutions for reproduction.

Another possible improvement would be usage of different crossover functions, such as the uniform allele crossover, which randomly changes a value contained in a 'gene'. This could help increase diversity of the solutions and possibly improve the fitness of the offspring.


Conducting deeper analysis on the change in performance based on random seed, population size, tournament size, and mutation probability paramters would be extremely insightful given the time. We know that the increase of population size and tournament size would increase the number of fittest solutions selected for reproduction, however, it will reach a point where the search is less efficient due to the size of the possible solutions and the time it would take to filter through them. 

#### Resources

I did not use any in-text referencing throughout the coursework. However, I used plenty of online resources and similar studies to help build my solution and guide me through the coding process and they provided me with a deeper understanding of every step. Therefore, I will still attatch the links to the **main** resources I used for guidance: 

- https://osuva.uwasa.fi/bitstream/handle/10024/10326/Osuva_Amil_Mantere_2019a.pdf?sequence=2&isAllowed=y
- https://www.cs.mcgill.ca/~dprecup/courses/AI/Lectures/ai-lecture05.pdf
- https://www.codeproject.com/Articles/365553/8-Puzzle-solving-using-the-A-algorithm-using-Pytho
- https://faramira.com/solving-8-puzzle-problem-using-a-star-search/
- https://www.linkedin.com/pulse/nature-inspired-sudoku-solver-efstathios-chatzikyriakidis-1f/
- http://theory.stanford.edu/~amitp/GameProgramming/Heuristics.html
- https://github.sre.pub/jmhummel/8puzzle
- https://courses.cs.washington.edu/courses/cse473/12au/slides/lect3.pdf